In [434]:
import math as m
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import time
import matplotlib as ml
import matplotlib.pyplot as pp

In [428]:
n = 20
h = 1.0 / n
mu = 8 * n**2 * m.cos(m.pi / (2.0 * n))
M = 8 * n**2 * m.sin(m.pi / (2.0 * n))

tau1 = (M + mu) / 2.0 - ((M - mu) / 2.0) * m.cos(m.pi / (2.0))
tau2 = (M + mu) / 2.0 - ((M - mu) / 2.0) * m.cos((m.pi * 3.0) / (4.0))

tau1 = 1.0 / tau1
tau2 = 1.0 / tau2

print("m = ", mu, " M = ", M)
print("tau1 = ", tau1, " tau2 = ", tau2)

eps = 0.001

m =  3190.1354679460096  M =  251.06910632910382
tau1 =  0.000581191834670654  tau2 =  0.0014673828457749147


In [429]:
def SetMatrix(A, n):
#     print(list(range(0, n)) + list(range(n, 2* n)) + list(range(n**2 - 2 * n, n**2 - n)) + list(range(n**2 - n, n**2)))
    for i in range(n**2):
        for j in range(n**2):
            if i == j:
                A[i][j] = 4  
            if j == i + 1 and i % n == 0:
                A[i][j] = -2
            elif j == i - 1 and i % n == n - 1:
                A[i][j] = -2
            elif (j == i - 1 or j == i + 1) and not ((i % n == n - 1 and j == i + 1) or (i % n == 0 and j == i - 1)):
                A[i][j] = -1
            elif i in list(range(0, n)) and j == i + n:
                A[i][j] = -2 
            elif i in list(range(n**2 - n, n**2)) and j == i - n:
                A[i][j] = -2
            elif (j == i - n or j == i + n):
                A[i][j] = -1 
#             if i % n == n - 1 and j == i + 1:
#                 A[i][j] = 0
#             if i % n == 0 and j == i - 1:
#                 A[i][j] = 0
                
def SetF(f, n):
    x = np.linspace(0.0, 1.0, num=n, endpoint=False)
    y = np.linspace(0.0, 1.0, num=n, endpoint=False)
    for i in range(n):
        for j in range(n):
            f[i * n + j] = m.cos(m.pi * x[i] * 6) * m.cos(m.pi * y[j] * 3) / n**2
            
def A_val(i, j, n):
    h = 1 #/ n
    a = 0
    if i == j:
        a = 4/h**2   
    if j == i + 1 and i % n == 0:
        a = -2/h**2  
    elif j == i - 1 and i % n == n - 1:
        a = -2/h**2  
    elif j == i - 1 or j == i + 1:
        a = -1/h**2 
    elif i in list(range(0, n)) and j == i + n:
        a = -2/h**2 
    elif i in list(range(n**2 - n, n**2)) and j == i - n:
        a = -2/h**2 
    elif (j == i - n or j == i + n):
        a = -1/h**2 
    if i % n == n - 1 and j == i + 1:
        a = 0
    if i % n == 0 and j == i - 1:
        a = 0
    return a
                
                
                
                
A = np.zeros( (n**2, n**2) )
f = np.zeros( n**2 )
x = np.ones( n**2 )
x_next = np.zeros( n**2 )
SetMatrix(A, n)
SetF(f, n)

# for i in range(n**2):
#     for j in range(n**2):       
#         print('{:.0f} '.format(A[i][j]), end=' ')
#     print('')
    
error = LA.norm(x - x_next)
# print("Error >> ", error)
# A

In [430]:
w = eigh(A, eigvals_only=True)
M = max(w)
mu = min(w)

tau1 = (M + mu) / 2.0 + ((M - mu) / 2.0) * m.cos(m.pi / (2.0 * n**2))
tau2 = (M + mu) / 2.0 + ((M - mu) / 2.0) * m.cos((m.pi * 3.0) / (2.0 * n**2))

tau1 = 1.0 / tau1
tau2 = 1.0 / tau2

print(mu)
M

-0.6188021526338999


8.6188021526339

In [433]:
error = LA.norm(x - x_next)
aii = 0
cij = 0
d = 0

i_cur = 0
j_cur = 0
step = -1

start_time = time.time()
eps = 0.0001

while(error > eps):
    for s in range(n):
        for k in range(n):
            step = 0
            x_next[s * n + k] = 0
#             aii = A_val(s * n + k, s * n + k, n)
            aii = A[s * n + k][s * n + k]
            d = f[s * n + k] / aii
            for i in range(n):
                i_cur = i
                for j in range(n):
                    j_cur = j
#                     aij = A_val(s * n + k, i * n + j, n)
                    aij = A[s * n + k][i * n + j]
                    if s * n + k == i * n + j:
                        cij = 0
                    else:
                        cij = (-aij/aii) 
                    x_next[s * n + k] += cij * x_next[i * n + j]
            for i in range(i_cur + 1, n):
                for j in range(j_cur + 1, n):
#                     aij = A_val(s * n + k, i * n + j, n)
                    aij = A[s * n + k][i * n + j]
                    if s * n + k == i * n + j:
                        cij = 0
                    else:
                        cij = (-aij/aii) #sss 
                    x_next[s * n + k] += cij * x[i * n + j]
            x_next[s * n + k] += d
            
    error = LA.norm(x - x_next)
    print("Error >> ", error)
    x = np.copy(x_next)
    
print("--- %s seconds ---" % (time.time() - start_time))
# print("|Ax - f| = ", LA.norm(np.dot(A, x) - f))

--- 0.0001971721649169922 seconds ---


In [381]:
LA.norm(np.dot(A, x) - f)

0.0018785134874500149

In [386]:
x1 = np.ones( n**2 )
e = np.zeros( (n**2, n**2) )
x_next = np.zeros( n**2 )
error = LA.norm(x1 - x_next)
step = 0
tau = 0

for i in range(n**2):
    e[i][i] = 1.0
    x1[i] += i * 0.002

tau = tau1
# print(e - tau * A)
# np.dot(A, x)

def mult(A, x, n):
    res = np.zeros( n**2 )
    
    for i in range(n**2):
        for j in range(n**2):
            res[i] += A[i][j] * x[j]
            
    return res

x1

array([1.   , 1.002, 1.004, ..., 4.194, 4.196, 4.198])

In [408]:
start_time = time.time()

while(error > eps):
    if(step % 2 == 0):
        tau = tau1
    else:
        tau = tau2
    print('tau = ', tau)
    C = e - tau * A
    x_next = np.dot(C, x1) + tau * f
    error = LA.norm(np.dot(A, x1) - f)
    print("Error >> ", error, " x_next:")
#     print(x_next)
#     print("x:")
#     print(x)
#     break
    x1 = np.copy(x_next)
    step += 1

print("--- %s seconds ---" % (time.time() - start_time))
print("|Ax - f| = ", LA.norm(np.dot(A, x1) - f))

tau =  0.11602543374882626
Error >>  0.08916742320971711  x_next:
tau =  0.11602567346431139
Error >>  0.08910031624193539  x_next:
tau =  0.11602543374882626
Error >>  0.08903325970754025  x_next:
tau =  0.11602567346431139
Error >>  0.08896625384515065  x_next:
tau =  0.11602543374882626
Error >>  0.08889929833924676  x_next:
tau =  0.11602567346431139
Error >>  0.08883239342809006  x_next:
tau =  0.11602543374882626
Error >>  0.08876553879664148  x_next:
tau =  0.11602567346431139
Error >>  0.0886987346828168  x_next:
tau =  0.11602543374882626
Error >>  0.08863198077205425  x_next:
tau =  0.11602567346431139
Error >>  0.088565277301919  x_next:
tau =  0.11602543374882626
Error >>  0.08849862395833023  x_next:
tau =  0.11602567346431139
Error >>  0.08843202097850163  x_next:
tau =  0.11602543374882626
Error >>  0.08836546804883241  x_next:
tau =  0.11602567346431139
Error >>  0.08829896540618512  x_next:
tau =  0.11602543374882626
Error >>  0.08823251273743991  x_next:
tau =  0.1160

Error >>  0.08109795104109212  x_next:
tau =  0.11602567346431139
Error >>  0.08103692254538812  x_next:
tau =  0.11602543374882626
Error >>  0.08097593987793275  x_next:
tau =  0.11602567346431139
Error >>  0.080915003256008  x_next:
tau =  0.11602543374882626
Error >>  0.08085411239295145  x_next:
tau =  0.11602567346431139
Error >>  0.08079326750572541  x_next:
tau =  0.11602543374882626
Error >>  0.08073246830810298  x_next:
tau =  0.11602567346431139
Error >>  0.08067171501671846  x_next:
tau =  0.11602543374882626
Error >>  0.08061100734578003  x_next:
tau =  0.11602567346431139
Error >>  0.08055034551160056  x_next:
tau =  0.11602543374882626
Error >>  0.08048972922882262  x_next:
tau =  0.11602567346431139
Error >>  0.08042915871343244  x_next:
tau =  0.11602543374882626
Error >>  0.08036863368050395  x_next:
tau =  0.11602567346431139
Error >>  0.08030815434570633  x_next:
tau =  0.11602543374882626
Error >>  0.08024772042454192  x_next:
tau =  0.11602567346431139
Error >>  0.

Error >>  0.07375918632640922  x_next:
tau =  0.11602567346431139
Error >>  0.07370368275958918  x_next:
tau =  0.11602543374882626
Error >>  0.07364822085660523  x_next:
tau =  0.11602567346431139
Error >>  0.07359280081518342  x_next:
tau =  0.11602543374882626
Error >>  0.07353742237473299  x_next:
tau =  0.11602567346431139
Error >>  0.07348208573268797  x_next:
tau =  0.11602543374882626
Error >>  0.07342679062885014  x_next:
tau =  0.11602567346431139
Error >>  0.07337153726035743  x_next:
tau =  0.11602543374882626
Error >>  0.07331632536740304  x_next:
tau =  0.11602567346431139
Error >>  0.07326115514682949  x_next:
tau =  0.11602543374882626
Error >>  0.0732060263392226  x_next:
tau =  0.11602567346431139
Error >>  0.07315093914113352  x_next:
tau =  0.11602543374882626
Error >>  0.0730958932935356  x_next:
tau =  0.11602567346431139
Error >>  0.07304088899268274  x_next:
tau =  0.11602543374882626
Error >>  0.07298592597994537  x_next:
tau =  0.11602567346431139
Error >>  0.

Error >>  0.06708471102313958  x_next:
tau =  0.11602567346431139
Error >>  0.06703423095920935  x_next:
tau =  0.11602543374882626
Error >>  0.06698378878160494  x_next:
tau =  0.11602567346431139
Error >>  0.06693338467021542  x_next:
tau =  0.11602543374882626
Error >>  0.06688301838807958  x_next:
tau =  0.11602567346431139
Error >>  0.06683269011481338  x_next:
tau =  0.11602543374882626
Error >>  0.06678239961381552  x_next:
tau =  0.11602567346431139
Error >>  0.06673214706443073  x_next:
tau =  0.11602543374882626
Error >>  0.06668193223041266  x_next:
tau =  0.11602567346431139
Error >>  0.06663175529083858  x_next:
tau =  0.11602543374882626
Error >>  0.06658161600982188  x_next:
tau =  0.11602567346431139
Error >>  0.06653151456616511  x_next:
tau =  0.11602543374882626
Error >>  0.06648145072433634  x_next:
tau =  0.11602567346431139
Error >>  0.06643142466287555  x_next:
tau =  0.11602543374882626
Error >>  0.06638143614660097  x_next:
tau =  0.11602567346431139
Error >>  

Error >>  0.06092249962373451  x_next:
tau =  0.11602567346431139
Error >>  0.06087665693789552  x_next:
tau =  0.11602543374882626
Error >>  0.060830848655101806  x_next:
tau =  0.11602567346431139
Error >>  0.060785074938728534  x_next:
tau =  0.11602543374882626
Error >>  0.06073933557361272  x_next:
tau =  0.11602567346431139
Error >>  0.06069363072288422  x_next:
tau =  0.11602543374882626
Error >>  0.06064796017170265  x_next:
tau =  0.11602567346431139
Error >>  0.06060232408295418  x_next:
tau =  0.11602543374882626
Error >>  0.06055672224211988  x_next:
tau =  0.11602567346431139
Error >>  0.060511154811845644  x_next:
tau =  0.11602543374882626
Error >>  0.06046562157793035  x_next:
tau =  0.11602567346431139
Error >>  0.060420122702776184  x_next:
tau =  0.11602543374882626
Error >>  0.060374657972507344  x_next:
tau =  0.11602567346431139
Error >>  0.06032922754927989  x_next:
tau =  0.11602543374882626
Error >>  0.06028383121954073  x_next:
tau =  0.11602567346431139
Error

Error >>  0.05540972321903829  x_next:
tau =  0.11602567346431139
Error >>  0.055368028939189816  x_next:
tau =  0.11602543374882626
Error >>  0.055326365947992895  x_next:
tau =  0.11602567346431139
Error >>  0.0552847343940515  x_next:
tau =  0.11602543374882626
Error >>  0.0552431340816802  x_next:
tau =  0.11602567346431139
Error >>  0.05520156515925627  x_next:
tau =  0.11602543374882626
Error >>  0.0551600274313876  x_next:
tau =  0.11602567346431139
Error >>  0.055118521046231925  x_next:
tau =  0.11602543374882626
Error >>  0.05507704580868869  x_next:
tau =  0.11602567346431139
Error >>  0.05503560186669401  x_next:
tau =  0.11602543374882626
Error >>  0.054994189025440086  x_next:
tau =  0.11602567346431139
Error >>  0.05495280743264152  x_next:
tau =  0.11602543374882626
Error >>  0.054911456893782976  x_next:
tau =  0.11602567346431139
Error >>  0.054870137556357404  x_next:
tau =  0.11602543374882626
Error >>  0.054828849226142264  x_next:
tau =  0.11602567346431139
Error 

Error >>  0.05047173168920916  x_next:
tau =  0.11602567346431139
Error >>  0.050433753186572644  x_next:
tau =  0.11602543374882626
Error >>  0.05039580318365432  x_next:
tau =  0.11602567346431139
Error >>  0.050357881815820384  x_next:
tau =  0.11602543374882626
Error >>  0.050319988904824184  x_next:
tau =  0.11602567346431139
Error >>  0.05028212458582677  x_next:
tau =  0.11602543374882626
Error >>  0.05024428868085338  x_next:
tau =  0.11602567346431139
Error >>  0.05020648132486  x_next:
tau =  0.11602543374882626
Error >>  0.050168702340138135  x_next:
tau =  0.11602567346431139
Error >>  0.05013095186144303  x_next:
tau =  0.11602543374882626
Error >>  0.05009322971132951  x_next:
tau =  0.11602567346431139
Error >>  0.050055536024355446  x_next:
tau =  0.11602543374882626
Error >>  0.05001787062334305  x_next:
tau =  0.11602567346431139
Error >>  0.04998023364264165  x_next:
tau =  0.11602543374882626
Error >>  0.04994262490534335  x_next:
tau =  0.11602567346431139
Error >>

KeyboardInterrupt: 